# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 18th 2025

**Student Name**: Aura Melina Gutierrez Jimenez (Ing. Sistemas Computacionales)

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data sources (Files)") \
    .master("spark://b1ca502cde8a:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/19 14:19:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pcamarillor.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

In [4]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/airline/")

df_airlines.show(n=5)

[Stage 0:>                                                          (0 + 1) / 1]

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

## Drop unnecessary columns.

In [5]:
df_airlines = df_airlines.drop("flight", "days_left")

## Count how many null values the dataset has before/after the cleaning process.

In [7]:
from pyspark.sql.functions import col, when, count, isnull
# Get number of null values before cleaning
print("Number of records before cleaning:", df_airlines.count())
df_airlines.select([count(when(isnull(col(c.name)), c.name)).alias(c.name) for c in df_airlines.schema]).show()

# Clean the dataset
airlines_clean = df_airlines.dropna()

# Get number of null values after cleaning
print("Number of records after cleaning:", airlines_clean.count())
airlines_clean.select([count(when(isnull(col(c.name)), c.name)).alias(c.name) for c in airlines_clean.schema]).show()

Number of records before cleaning: 300153


+-----+-------+-----------+--------------+-----+------------+----------------+-----+--------+-----+
|index|airline|source_city|departure_time|stops|arrival_time|destination_city|class|duration|price|
+-----+-------+-----------+--------------+-----+------------+----------------+-----+--------+-----+
|    0|      0|          0|             0|    0|           0|               0|    0|       0|    0|
+-----+-------+-----------+--------------+-----+------------+----------------+-----+--------+-----+



Number of records after cleaning: 300153


[Stage 13:=============================>                            (1 + 1) / 2]

+-----+-------+-----------+--------------+-----+------------+----------------+-----+--------+-----+
|index|airline|source_city|departure_time|stops|arrival_time|destination_city|class|duration|price|
+-----+-------+-----------+--------------+-----+------------+----------------+-----+--------+-----+
|    0|      0|          0|             0|    0|           0|               0|    0|       0|    0|
+-----+-------+-----------+--------------+-----+------------+----------------+-----+--------+-----+

